In [1]:
import pandas as pd
from math import sqrt
import numpy as np

In [3]:
books = pd.read_csv("BX-Books.csv", encoding = 'ISO-8859-1', on_bad_lines='skip', sep = ';')
rating = pd.read_csv("BX-Book-Ratings.csv", encoding = 'ISO-8859-1', on_bad_lines='skip', sep = ';')
print(books.info())
print(rating.info())

/var/folders/38/zbfzn11s771cr66zl8dvyv2w0000gn/T/ipykernel_58131/2385365961.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("BX-Books.csv", encoding = 'ISO-8859-1', on_bad_lines='skip', sep = ';')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2),

In [4]:
userInput = [{'Book-Title' : 'Jane Doe', 'Book-Rating' : 8}, 
             {'Book-Title' : 'The Middle Stories', 'Book-Rating' : 10},
             {'Book-Title' : 'Nights Below Station Street', 'Book-Rating' : 3},
             {'Book-Title' : 'Wild Animus', 'Book-Rating' : 5},
             {'Book-Title' : 'Airframe', 'Book-Rating' : 1}]

inputBooks = pd.DataFrame(userInput)
print(inputBooks)

                    Book-Title  Book-Rating
0                     Jane Doe            8
1           The Middle Stories           10
2  Nights Below Station Street            3
3                  Wild Animus            5
4                     Airframe            1


In [5]:
inputISBN = books[books['Book-Title'].isin(inputBooks['Book-Title'].tolist())]
inputBooks = pd.merge(inputISBN, inputBooks)
inputBooks = inputBooks[['ISBN', 'Book-Title', 'Book-Rating']]
print(inputBooks)

         ISBN                   Book-Title  Book-Rating
0  0771074670  Nights Below Station Street            3
1  0887841740           The Middle Stories           10
2  1552041778                     Jane Doe            8
3  1551665107                     Jane Doe            8
4  0971880107                  Wild Animus            5
5  0345402871                     Airframe            1
6  0679446486                     Airframe            1
7  0613072103                     Airframe            1
8  0679452664                     Airframe            1
9  0679457976                     Airframe            1


In [6]:
userSubset = rating[rating['ISBN'].isin(inputBooks['ISBN'].tolist())]
print(userSubset.groupby('ISBN').count())

            User-ID  Book-Rating
ISBN                            
0345402871      207          207
0613072103        1            1
0679446486      120          120
0679452664        3            3
0679457976        1            1
0771074670        6            6
0887841740        2            2
0971880107     2502         2502
1551665107       22           22
1552041778        1            1


In [7]:
userSubsetGroup = userSubset.groupby(['User-ID'])

def take_5_elem(x):
    return len(x[1])

userSubsetGroup = sorted(userSubsetGroup, key=take_5_elem, reverse=True)

userSubsetGroup = userSubsetGroup[0:100]
print(userSubsetGroup[0:5])

[(11676,        User-ID        ISBN  Book-Rating
47675    11676  0345402871            0
52913    11676  0679446486            0
54043    11676  0771074670            0
55317    11676  0971880107            6), (8,       User-ID        ISBN  Book-Rating
9573        8  0771074670            0
9575        8  0887841740            5
9576        8  1552041778            5), (35859,         User-ID        ISBN  Book-Rating
161577    35859  0679452664            0
162665    35859  0971880107            0
162695    35859  1551665107            0), (166596,         User-ID        ISBN  Book-Rating
683887   166596  0345402871            0
684636   166596  0771074670            0
684808   166596  1551665107            0), (167471,         User-ID        ISBN  Book-Rating
687071   167471  0345402871            0
687325   167471  0679446486           10
687450   167471  0971880107            0)]


In [8]:
pearsonCorrelationDict = {}

for name, group in userSubsetGroup:
    group = group.sort_values(by='ISBN')
    inputBooks = inputBooks.sort_values(by='ISBN')
    nRatings = len(group)
    temp_df = inputBooks[inputBooks['ISBN'].isin(group['ISBN'].tolist())]
    tempRatingList = temp_df['Book-Rating'].tolist()
    tempGroupList = group['Book-Rating'].tolist()
    
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [9]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['User-ID'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
print(pearsonDF.head())

   similarityIndex  User-ID
0         0.870388    11676
1         0.960769        8
2         0.000000    35859
3         0.000000   166596
4        -0.500000   167471


In [10]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
print(topUsers.head())

    similarityIndex  User-ID
35              1.0    93047
47              1.0   167800
67              1.0   247201
33              1.0    88693
44              1.0   161872


In [11]:
topUsersRating=topUsers.merge(rating, left_on='User-ID', right_on='User-ID', how='inner')
print(topUsersRating.head(100))

    similarityIndex  User-ID        ISBN  Book-Rating
0               1.0    93047  0001056107            8
1               1.0    93047  0006392555            7
2               1.0    93047  0006548180            8
3               1.0    93047  0020093616            8
4               1.0    93047  0020404808            0
..              ...      ...         ...          ...
95              1.0    93047  0070657599            7
96              1.0    93047  0099351617            0
97              1.0    93047  0099592207            0
98              1.0    93047  0140034013            0
99              1.0    93047  0140037969            8

[100 rows x 4 columns]


In [12]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['Book-Rating']
print(topUsersRating.head())

   similarityIndex  User-ID        ISBN  Book-Rating  weightedRating
0              1.0    93047  0001056107            8             8.0
1              1.0    93047  0006392555            7             7.0
2              1.0    93047  0006548180            8             8.0
3              1.0    93047  0020093616            8             8.0
4              1.0    93047  0020404808            0             0.0


In [13]:
tempTopUsersRating = topUsersRating.groupby('ISBN').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
print(tempTopUsersRating.head())

               sum_similarityIndex  sum_weightedRating
ISBN                                                  
 9022906116               0.870388            6.092718
0 7336 1053 6             0.870388            0.000000
0000000000                0.870388            7.833495
00000000000               0.870388            6.963106
0001055666                0.870388            7.833495


In [14]:
recommendation_df = pd.DataFrame()

recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['ISBN'] = tempTopUsersRating.index
print(recommendation_df.head(10))

               weighted average recommendation score           ISBN
ISBN                                                               
 9022906116                                      7.0     9022906116
0 7336 1053 6                                    0.0  0 7336 1053 6
0000000000                                       9.0     0000000000
00000000000                                      8.0    00000000000
0001055666                                       9.0     0001055666
0001056107                                       8.0     0001056107
000123207X                                       NaN     000123207X
0001232088                                       NaN     0001232088
0001837397                                       0.0     0001837397
0001841572                                       0.0     0001841572


In [15]:
recommendation_df = recommendation_df.sort_values(by = 'weighted average recommendation score', ascending = False)
print(recommendation_df)

               weighted average recommendation score           ISBN
ISBN                                                               
0446527130                                      10.0     0446527130
0373712006                                      10.0     0373712006
0373790058                                      10.0     0373790058
0140265988                                      10.0     0140265988
0751520195                                      10.0     0751520195
...                                              ...            ...
9783499191411                                    NaN  9783499191411
9997407210                                       NaN     9997407210
B000023VWQ                                       NaN     B000023VWQ
B0000DAPP1                                       NaN     B0000DAPP1
THECATASTROPH                                    NaN  THECATASTROPH

[33337 rows x 2 columns]


In [16]:
recommended_book = books.loc[books['ISBN'].isin(recommendation_df['ISBN'])]

recommended_book = recommended_book.loc[~recommended_book.ISBN.isin(userSubset['ISBN'])]

print(recommended_book)

              ISBN                                         Book-Title  \
1       0002005018                                       Clara Callan   
2       0060973129                               Decision in Normandy   
3       0374157065  Flu: The Story of the Great Influenza Pandemic...   
4       0393045218                             The Mummies of Urumchi   
5       0399135782                             The Kitchen God's Wife   
...            ...                                                ...   
271299  0801042615             Reservoir Road Adventure (Voyager Ser)   
271300  0803267177           Wigwam Evenings: Sioux Folk Tales Retold   
271301  0843101083                     Off-The-Wall (Mad Libs, No. 6)   
271302  0875346235  The Dream Violin and Other Stories of Families...   
271303  0898860792                               Cross-country skiing   

                    Book-Author Year-Of-Publication  \
1          Richard Bruce Wright                2001   
2            